In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
from glob import glob
import os
import gc
import scipy.stats
import pdb
iters = 1000
nrecs= [10,50,100]
# nrecs= [100]
n_inits = [10,50,100]
# n_inits = [100]
# nrecs= [10,100]
# n_inits = [10,100]
resdir = 'results_add/'
if not os.path.exists(resdir+'summary/'):
    os.mkdir(resdir+'summary/')
knowledgebase='sklearn'
# knowledgebase='hibachi'
# from tqdm import tqdm
for n_init in n_inits:
    for n_rec in nrecs:
        frames = []
        print(50*'=','\n','n_init:',n_init,'n_rec:',n_rec,'\n',50*'=')
        experiment = (resdir +'experiment_'+knowledgebase+'*_ninit-'+str(n_init)+'_*nrecs-'+str(n_rec)+
                      '_*iters-'+str(iters)+'*csv')
        print(experiment)
        savename = experiment.split('/')[-1].replace('*','').split('csv')[0]
        print('savename:',savename)
        batch_size = max(round(len(glob(experiment))/10),1000)
        print('experiment size:',len(glob(experiment)))
        print('batch_size:',batch_size)
        batch = batch_size 
        df_exp = pd.DataFrame()
        for i,f in enumerate(glob(experiment)):
#             print(f)
            df = pd.read_csv(f)
#             df = df[['n_recs','n_init','iteration','delta_bal_accuracy','recommender','ranking','iters',
#                     'ml-rec','score-rec','bal_accuracy','best_algorithm','trial']]
            # store dataframe of mean, median, 
            frames.append(df)
            if i > batch or i == len(glob(experiment))-1:
                print('writing batch','batch=',batch)
                batch += batch_size
                df_exp_batch = pd.concat(frames)
                df_exp = df_exp.append(df_exp_batch)
                del df_exp_batch
                frames = []
            gc.collect()
#         pdb.set_trace()
        ##########
        # summary stats
        ##########
        print('loaded',len(df_exp),'experiments')
        print('columns:',df_exp.columns)
        print('recommenders:',df_exp.recommender.unique())
        print('trials:',len(df_exp.trial.unique()))
        print('datasets:',len(df_exp.dataset.unique()))
        df_sum = pd.DataFrame()
        for c in ['delta_bal_accuracy','ranking']:
            df_sum[c+'_mean'] = df_exp.groupby(['iteration','recommender'])[c].mean()
            df_sum[c+'_median'] = df_exp.groupby(['iteration','recommender'])[c].median()
            df_sum[c+'_std'] = df_exp.groupby(['iteration','recommender'])[c].std()
            df_sum[c+'_min'] = df_exp.groupby(['iteration','recommender'])[c].min()
            df_sum[c+'_se'] = df_exp.groupby(['iteration','recommender'])[c].apply(scipy.stats.sem)
            df_sum[c+'_q1'] = df_exp.groupby(['iteration','recommender'])[c].quantile(0.25)
            df_sum[c+'_q3'] = df_exp.groupby(['iteration','recommender'])[c].quantile(0.75)
        # get success rate
#         pdb.set_trace()    
        
        df_sum['rank_1_ratio'] = df_exp.groupby(['iteration','recommender','trial'])['ranking'].value_counts(
                normalize=True)[:,:,:,1.0].unstack('trial').median()
        df_sum['iters'] = iters
        df_sum['n_recs'] = n_rec
        df_sum['n_init'] = n_init
        df_sum.drop_duplicates(inplace=True)
        print('saving summary stats to',resdir +savename+'_summary.csv')
        df_sum.to_csv(resdir +'summary/'+savename+'_summary.csv')
        
#         #############
#         # save a modified confusion matrix that shows where mistakes are made
#         #############
# #         import seaborn as sns
# #         import matplotlib.pyplot as plt
# #         %matplotlib inline
        ml_options = df_exp['ml-rec'].unique()
#         for rec,dfg in df_exp.groupby(['recommender']):
#             df = pd.DataFrame(np.zeros((len(ml_options),len(ml_options))),
#                               dtype=np.float,
#                               columns=ml_options, index=ml_options) 
#             for option in ml_options:
#                 for option in ml_options:
#                     df[option][option] = 0
#             for _,row in dfg.iterrows():
#                 if row['ranking'] == 1: 
#                     df[row['ml-rec']][row['ml-rec']] += 1
#                 else:
#                     nbest = float(len(row['best_algorithm'].split('|')))
# #                     print('nbest:',nbest) 
#                     for mlr in row['best_algorithm'].split('|'):
# #                         print('filling in',row['ml-rec'],',',mlr)
#                         df[row['ml-rec']][mlr] += 1/nbest
#             print('saving',resdir +'summary/'+savename
#                                 +'_confusion_matrix_'+rec+'.csv')
#             df.to_csv(resdir +'summary/'+savename
#                                 +'_confusion_matrix_'+rec+'.csv')
#         plt.show()
#         assert(False)
        #############
        # save counts of each ML recommendation made by each recommender 
        #############
        heat_frames = []
        for (it,rec),dfg in df_exp.groupby(['iteration','recommender']):
            df = {} 
            df['iteration'] = it
            df['recommender'] = rec
            for option in ml_options:
                df[option] = 0
            for ml_rec in dfg['ml-rec']:
                df[ml_rec] += 1
            heat_frames.append(df)
        # add "best" as a recommender
        for it,dfg in df_exp.groupby(['iteration']):
            df = {} 
            df['iteration'] = it
            df['recommender'] = 'best'
            for option in ml_options:
                df[option] = 0
            for ml_rec in dfg['best_algorithm']:
                for mlr in ml_rec.split('|'):
                    df[mlr] += 1
            heat_frames.append(df)
        df_heatmap = pd.DataFrame.from_records(heat_frames).set_index('iteration')
        print('saving',resdir +'summary/'+savename+'_heatmap.csv')
        df_heatmap.to_csv(resdir +'summary/'+savename+'_heatmap.csv')
#         pdb.set_trace()
        
print('done!')